# Trainer

> Scripts to train the models

In [6]:
#| default_exp trainer

In [7]:
#| export
#| hide
import torch
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning import Trainer
from sklearn.model_selection import train_test_split

In [8]:
#| export
def train_model(model, data, labels, params):
    """
    Trains the given model using the provided data and configuration.

    Parameters:
    -----------
    model : pl.LightningModule
        The model to be trained.
    
    data : torch.Tensor or np.array
        The input data (features) to train the model.
    
    labels : torch.Tensor or np.array
        The corresponding labels for the input data.
    
    config : dict
        Configuration dictionary containing training parameters like batch_size, epochs, val_split, etc.
    """
    
    # Validate params keys with defaults
    epochs = params.get('epochs', 10)  # Default to 10 if not specified
    batch_size = params.get('batch_size', 32)  # Default batch size
    val_split = params.get('val_split', 0.1)  # Default validation split

    print(f"Configured epochs: {epochs}, batch_size: {batch_size}, val_split: {val_split}")

    if not isinstance(data, torch.Tensor):
        print('no')
    # Convert data to tensors if not already
    data = torch.tensor(data, dtype=torch.float32) if not isinstance(data, torch.Tensor) else data
    labels = torch.tensor(labels, dtype=torch.long) if not isinstance(labels, torch.Tensor) else labels

    # Split the data and create dataloaders
    if val_split > 0:
        X_train, X_val, y_train, y_val = train_test_split(
            data, labels,
            test_size=val_split,
            random_state=42
        )
        train_dataset = TensorDataset(X_train)
        val_dataset = TensorDataset(X_val)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=15)
    else:
        X_train, y_train = data, labels
        val_dataloader = None

    train_dataset = TensorDataset(X_train)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=15)

    # Initialize PyTorch Lightning Trainer with GPU settings
    trainer = Trainer(
        max_epochs=epochs,
        log_every_n_steps=10,
        devices="auto",  # Automatically use available GPUs
        accelerator="auto"  # Automatically select the best accelerator (GPU or CPU)
    )

    print("Starting training...")
    # Train the model
    if val_dataloader:
        trainer.fit(model, train_dataloader, val_dataloader)
    else:
        trainer.fit(model, train_dataloader)

    # Return metrics after training is complete
    print("Training complete. Here are the metrics: ")
    return model.metrics

In [9]:
#| export
def train_model_with_explicit_validation(model, train_data, train_labels, val_data, val_labels, config):
    """
    Trains the given model using the provided data and configuration, with explicitly set validation data.

    Parameters:
    -----------
    model : pl.LightningModule
        The model to be trained.
    
    train_data : torch.Tensor or np.array
        The input data (features) for training.
    
    train_labels : torch.Tensor or np.array
        The corresponding labels for the training data.
    
    val_data : torch.Tensor or np.array
        The input data (features) for validation.
    
    val_labels : torch.Tensor or np.array
        The corresponding labels for the validation data.
    
    config : dict
        Configuration dictionary containing training parameters like batch_size, epochs, etc.
    """

    # Create PyTorch datasets
    train_dataset = TensorDataset(torch.tensor(train_data), torch.tensor(train_labels))
    val_dataset = TensorDataset(torch.tensor(val_data), torch.tensor(val_labels))

    # Create DataLoaders
    train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'])

    # Initialize PyTorch Lightning Trainer
    trainer = Trainer(
        max_epochs=config['epochs'],
        log_every_n_steps=10,
        gpus=1 if torch.cuda.is_available() else 0
    )

    # Train the model
    trainer.fit(model, train_dataloader, val_dataloader)

    # Optionally return metrics (assuming they are logged within the model)
    return model.metrics

In [10]:
#| hide
import nbdev; nbdev.nbdev_export()